In [1]:
import pandas as pd
import random
from time import sleep
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
import pickle
from nltk.tokenize import word_tokenize
from sklearn.decomposition import NMF, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc, fbeta_score
from sklearn.model_selection import learning_curve, KFold, GridSearchCV, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text
from sklearn.preprocessing import normalize
import re
from unidecode import unidecode

import matplotlib.style as style
%matplotlib inline
plt.style.use('ggplot')

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open('data/cleaned_df.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [3]:
df.head()

,song_owner,song_title,song_element,verse,artist
0,Childish Gambino,This Is America,[Bridge: Childish Gambino & Young Thug],"We just wanna party, Party just for you, We ju...",Childish Gambino
1,Childish Gambino,This Is America,[Chorus: Childish Gambino],"This is America, Don't catch you slippin' now,...",Childish Gambino
2,Childish Gambino,This Is America,"[Verse 1: Childish Gambino, Blocboy JB, Slim J...","This is America (skrrt, skrrt, woo), Don't cat...",Childish Gambino
3,Childish Gambino,This Is America,[Refrain: Choir & Childish Gambino],"Ooh-ooh-ooh-ooh-ooh, tell somebody, You go tel...",Childish Gambino
4,Childish Gambino,This Is America,"[Chorus: Childish Gambino, Slim Jxmmi & Young ...","This is America (woo, ayy), Don't catch you sl...",Childish Gambino


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6304 entries, 0 to 6303
Data columns (total 5 columns):
song_owner      6304 non-null object
song_title      6304 non-null object
song_element    6304 non-null object
verse           6304 non-null object
artist          6304 non-null object
dtypes: object(5)
memory usage: 246.3+ KB


In [5]:
model_df = df.copy()

model_df = model_df[model_df['artist'] == 'Drake']

In [6]:
model_df['verse_words'] = model_df['verse'].copy()
model_df['verse_words'] = model_df['verse_words'].str.lower()
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.replace(',',''))
model_df['verse_words'] = model_df['verse_words'].apply(lambda x: x.split(' '))

In [7]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words
1282,Drake,Buried Alive (Interlude),[Intro],"If you was in a pine box, I would surely break...",Drake,"[if, you, was, in, a, pine, box, i, would, sur..."
1284,Drake,God's Plan,[Intro],"Yeah, they wishin' and wishin' and wishin' and...",Drake,"[yeah, they, wishin', and, wishin', and, wishi..."
1285,Drake,God's Plan,[Verse 1],"I been movin' calm, don't start no trouble wit...",Drake,"[i, been, movin', calm, don't, start, no, trou..."
1286,Drake,God's Plan,[Chorus 1],"God's plan, God's plan, I hold back, sometimes...",Drake,"[god's, plan, god's, plan, i, hold, back, some..."
1287,Drake,God's Plan,[Post-Chorus],"And still, bad things, It's a lot of bad thing...",Drake,"[and, still, bad, things, it's, a, lot, of, ba..."


In [8]:
from nltk.tokenize import word_tokenize
model_df['tokenize_verse'] = model_df['verse'].copy()
model_df['tokenize_verse'] = model_df['tokenize_verse'].str.lower()
model_df['tokenize_verse'] = model_df['tokenize_verse'].apply(lambda x: word_tokenize(x))

In [9]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
1282,Drake,Buried Alive (Interlude),[Intro],"If you was in a pine box, I would surely break...",Drake,"[if, you, was, in, a, pine, box, i, would, sur...","[if, you, was, in, a, pine, box, ,, i, would, ..."
1284,Drake,God's Plan,[Intro],"Yeah, they wishin' and wishin' and wishin' and...",Drake,"[yeah, they, wishin', and, wishin', and, wishi...","[yeah, ,, they, wishin, ', and, wishin, ', and..."
1285,Drake,God's Plan,[Verse 1],"I been movin' calm, don't start no trouble wit...",Drake,"[i, been, movin', calm, don't, start, no, trou...","[i, been, movin, ', calm, ,, do, n't, start, n..."
1286,Drake,God's Plan,[Chorus 1],"God's plan, God's plan, I hold back, sometimes...",Drake,"[god's, plan, god's, plan, i, hold, back, some...","[god, 's, plan, ,, god, 's, plan, ,, i, hold, ..."
1287,Drake,God's Plan,[Post-Chorus],"And still, bad things, It's a lot of bad thing...",Drake,"[and, still, bad, things, it's, a, lot, of, ba...","[and, still, ,, bad, things, ,, it, 's, a, lot..."


In [10]:
model_df['verse'] = model_df['verse'].str.lower()
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace("'",''))
model_df['verse'] = model_df['verse'].apply(lambda x: "".join([c if c.isalnum() else " " for c in x ]))
model_df['verse'] = model_df['verse'].apply(lambda x: x.replace('  ',' '))

In [11]:
stopwords = ["yeah", "uh", "huh", "aw", "oh", "la", "ayy",
            "na", "ha", "ay", "skrrt",
            "em", "bla", "woo", "wu", "ya", "wala", "woop", "wrarrr", "ooh",
            "ba", "duh", "du", "1", "2", "3", "4" ,"5" ,"6", "7", "8", "9", "0"]

for index, row in model_df.iterrows():
    querywords = row['verse'].split()

    resultwords  = [word for word in querywords if word.lower() not in stopwords]
    row['verse'] = ' '.join(resultwords)

model_df.verse[:5]

1282    if you was in a pine box i would surely break ...
1284    they wishin and wishin and wishin and wishin t...
1285    i been movin calm dont start no trouble with m...
1286    gods plan gods plan i hold back sometimes i wo...
1287    and still bad things its a lot of bad things t...
Name: verse, dtype: object

In [12]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
1282,Drake,Buried Alive (Interlude),[Intro],if you was in a pine box i would surely break ...,Drake,"[if, you, was, in, a, pine, box, i, would, sur...","[if, you, was, in, a, pine, box, ,, i, would, ..."
1284,Drake,God's Plan,[Intro],they wishin and wishin and wishin and wishin t...,Drake,"[yeah, they, wishin', and, wishin', and, wishi...","[yeah, ,, they, wishin, ', and, wishin, ', and..."
1285,Drake,God's Plan,[Verse 1],i been movin calm dont start no trouble with m...,Drake,"[i, been, movin', calm, don't, start, no, trou...","[i, been, movin, ', calm, ,, do, n't, start, n..."
1286,Drake,God's Plan,[Chorus 1],gods plan gods plan i hold back sometimes i wo...,Drake,"[god's, plan, god's, plan, i, hold, back, some...","[god, 's, plan, ,, god, 's, plan, ,, i, hold, ..."
1287,Drake,God's Plan,[Post-Chorus],and still bad things its a lot of bad things t...,Drake,"[and, still, bad, things, it's, a, lot, of, ba...","[and, still, ,, bad, things, ,, it, 's, a, lot..."


In [13]:
def get_words(df):
    all_words = []
    
    for index, row in df['verse'].iteritems():
        new_words = re.findall(r"\b[a-z']+\b", unidecode(row))
        all_words += new_words
        
    return all_words

In [14]:
model_df.head()

,song_owner,song_title,song_element,verse,artist,verse_words,tokenize_verse
1282,Drake,Buried Alive (Interlude),[Intro],if you was in a pine box i would surely break ...,Drake,"[if, you, was, in, a, pine, box, i, would, sur...","[if, you, was, in, a, pine, box, ,, i, would, ..."
1284,Drake,God's Plan,[Intro],they wishin and wishin and wishin and wishin t...,Drake,"[yeah, they, wishin', and, wishin', and, wishi...","[yeah, ,, they, wishin, ', and, wishin, ', and..."
1285,Drake,God's Plan,[Verse 1],i been movin calm dont start no trouble with m...,Drake,"[i, been, movin', calm, don't, start, no, trou...","[i, been, movin, ', calm, ,, do, n't, start, n..."
1286,Drake,God's Plan,[Chorus 1],gods plan gods plan i hold back sometimes i wo...,Drake,"[god's, plan, god's, plan, i, hold, back, some...","[god, 's, plan, ,, god, 's, plan, ,, i, hold, ..."
1287,Drake,God's Plan,[Post-Chorus],and still bad things its a lot of bad things t...,Drake,"[and, still, bad, things, it's, a, lot, of, ba...","[and, still, ,, bad, things, ,, it, 's, a, lot..."


In [15]:
get_words(model_df)

['if',
 'you',
 'was',
 'in',
 'a',
 'pine',
 'box',
 'i',
 'would',
 'surely',
 'break',
 'the',
 'lock',
 'id',
 'jump',
 'right',
 'in',
 'and',
 'fall',
 'asleep',
 'cause',
 'you',
 'are',
 'the',
 'death',
 'of',
 'me',
 'if',
 'you',
 'was',
 'in',
 'a',
 'pine',
 'box',
 'i',
 'would',
 'surely',
 'break',
 'the',
 'lock',
 'id',
 'jump',
 'right',
 'in',
 'and',
 'fall',
 'asleep',
 'cause',
 'you',
 'are',
 'the',
 'death',
 'of',
 'me',
 'they',
 'wishin',
 'and',
 'wishin',
 'and',
 'wishin',
 'and',
 'wishin',
 'they',
 'wishin',
 'on',
 'me',
 'yuh',
 'i',
 'been',
 'movin',
 'calm',
 'dont',
 'start',
 'no',
 'trouble',
 'with',
 'me',
 'tryna',
 'keep',
 'it',
 'peaceful',
 'is',
 'a',
 'struggle',
 'for',
 'me',
 'dont',
 'pull',
 'up',
 'at',
 'am',
 'to',
 'cuddle',
 'with',
 'me',
 'you',
 'know',
 'how',
 'i',
 'like',
 'it',
 'when',
 'you',
 'lovin',
 'on',
 'me',
 'i',
 'dont',
 'wanna',
 'die',
 'for',
 'them',
 'to',
 'miss',
 'me',
 'yes',
 'i',
 'see',
 'the

In [17]:
verses = list(model_df.verse.values)

In [18]:
verses[:10]

['if you was in a pine box i would surely break the lock id jump right in and fall asleep cause you are the death of me if you was in a pine box i would surely break the lock id jump right in and fall asleep cause you are the death of me',
 'they wishin and wishin and wishin and wishin they wishin on me yuh',
 'i been movin calm dont start no trouble with me tryna keep it peaceful is a struggle for me dont pull up at am to cuddle with me you know how i like it when you lovin on me i dont wanna die for them to miss me yes i see the things that they wishin on me hope i got some brothers that outlive me they gon tell the story shit was different with me',
 'gods plan gods plan i hold back sometimes i wont yuh i feel good sometimes i dont dont i finessed down weston road nessed might go down a g o d wait i go hard on southside g yuh wait i make sure that north side eat',
 'and still bad things its a lot of bad things that they wishin and wishin and wishin and wishin they wishin on me bad t

In [19]:
def make_word_dictionary(verses):
    following = {}
    count = {}
    for verse in verses:
        words = verse.split()
        for index, word in enumerate(words[:-1]):
            # ensure the word is in the dictionary
            if word not in following:
                following[word] = []
                count[word] = 0

            # append the next word
            following[word].append(words[index + 1])
            count[word] += 1

        if words:
            last_word = words[-1]
            if last_word not in following:
                following[last_word] = []
                count[last_word] = 0
                
    return count, following

In [20]:
count, following = make_word_dictionary(verses)

In [21]:
def create_bigrams(verses):
    bigrams = {}
    for words in verses:
        words = words.split()
        words = [x for x in words if x]
        for w1, w2, w3 in zip(words[:-2], words[1:-1], words[2:]):
            bigram = (w1,w2)
            if bigram not in bigrams:
                bigrams[bigram] = [w3]
            else:
                bigrams[bigram].append(w3)   
    return bigrams

In [22]:
verses_bigrams = create_bigrams(verses)

In [23]:
def generate_text_from_bigram(following, bigram, num_words, seeds):    
    all_bigrams = list(bigram.keys())
    current_bigram = random.choice(seeds)
    text = [current_bigram]
    for _ in range(num_words - 1):
        if current_bigram in all_bigrams:
            
            next_word = random.choice(bigram[current_bigram])
        else:
            if following[text[-1]]:
                next_word = random.choice(following[text[-1]])
            else:
                continue
        text.append(next_word)
        current_bigram = (current_bigram[1], next_word)
    return ' '.join(text)

In [24]:
seeds = ["bitch", "love", "drugs", "cash", "guns", "club", "friends", "paper", "liquor", "apple",
         "blue", "red", "streets", "people", "brother", "cap"]

generated_raps = []
for _ in range(0,100):
    wordlist = generate_text_from_bigram(following, verses_bigrams, 33, seeds)
    generated_raps.append(wordlist)
    print(wordlist)
    print('\n')

club like im faded faded f faded faded f faded f faded faded f faded f faded f faded f faded im feelin a way young nigga tryin to let new people in


love you ask me cash like johnny banks like ashley burnin like a statue pink champagne green ink in your zone looking like a damn shame that michael crabtree cant catch it people


blue moon since i left for me they wanna be anywhere they ask me my enemies eat keep gettin temperature checks they know they know i know you was never let down but


drugs weed addiction you got potential i cant say no how you supposed to forget im jewish i guess thats just me you wouldnt have worked this hard if you was livin at


paper like its my turn like advance green my true religion is money well we ll talk about pussy power she used to reintroduced to people i need you second guessing me jail


red lipstick and concealer devil in a fucked up the world it all just happen when i leave you alone be steady these the same intentions talking strippers and mo

friends never called you cause they think i belong right where i get it thats got all bitter im a stockholder private flights back home 40 days on the bench nigga we made


drugs weed addiction you got a roommate call me when you done laps roll around with all that lovey dovey shit im your brother shit all on my way girl let me hear


people dont got it like a motherfucking president to represent me nigga calm down tryna cool it all on my mind off the success and faded off of the key to your ex


people like all my niggas thats not on this side im not the same mistakes and i am have nada of me never meant wrong tried to sell weed give me all her


friends that unity all that other shit thats concernin me theres nothing realer just enough to pay the price is high i could be mine youll still be mine i control this you


people need me to call me over when its sunny you know i said i call for your imagination you should take it all in your ear thats me cant you see how


club instead its 